In [0]:
%run ../utils/common

In [0]:

dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_winare_auto_block_promotion_dc_current
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS

WITH free_goods AS (
    SELECT 
        a.*,
        CASE 
            WHEN TRIM(IFNULL(a.store_id, '') || IFNULL(a.supply_region, '')) <> '' 
                THEN TRIM(IFNULL(a.store_id, '') || IFNULL(a.supply_region, '')) 
            ELSE '1401' 
        END AS varkey
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_free_goods_promotion a
    WHERE (
        (a.valid_from - 7 <= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
         AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) <= a.valid_from + 7
         AND DATEDIFF(valid_to, valid_from) + 1 >= 7)
        OR
        (a.valid_from - 7 <= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
         AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) <= a.valid_from + 3
         AND DATEDIFF(valid_to, valid_from) + 1 < 7)
    )
),

free_goods_by_site AS (
    SELECT a.*
    FROM (
        SELECT
            COALESCE(a.store_id, st2.store_id, st1.store_id) AS dc_id,
            a.article AS product_id,
            a.vendor_id,
            a.valid_from,
            a.valid_to,
            a.promo_id,
            a.promo_theme,
            a.varkey
        FROM free_goods a
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store st1 ON a.varkey = '1401'
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store st2 ON a.varkey = st2.supply_region
    ) a
    INNER JOIN (
        SELECT DISTINCT dc_id
        FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_winare_dc_golive
    ) b ON a.dc_id = b.dc_id
),

promotion_price AS (
    SELECT *,
        CASE 
            WHEN TRIM(IFNULL(a.store_id, '') || IFNULL(a.supply_region, '')) <> '' 
                THEN TRIM(IFNULL(a.store_id, '') || IFNULL(a.supply_region, '')) 
            ELSE '1401' 
        END AS varkey
    FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_sellin_price_promotion_block a
),

promotion_price_by_site AS (
    SELECT a.*
    FROM (
        SELECT
            COALESCE(a.store_id, st2.store_id, st1.store_id) AS dc_id,
            a.product_id,
            LTRIM('0',a.vendor) AS vendor_id,
            a.price_from AS valid_from,
            a.price_to AS valid_to,
            a.promotion AS promo_id,
            a.promo_theme,
            a.varkey
        FROM promotion_price a
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store st1 ON a.varkey = '1401'
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store st2 ON a.varkey = st2.supply_region
    ) a
    INNER JOIN (
        SELECT DISTINCT dc_id
        FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_winare_dc_golive
    ) b ON a.dc_id = b.dc_id
)

SELECT *, 'FREE_GOOD' AS reason
FROM free_goods_by_site

UNION ALL

SELECT *, 
    CASE 
        WHEN LEFT(promo_theme, 1) = 'B' THEN 'PROMO_PRICE_BOKEM' 
        ELSE 'PROMO_PRICE' 
    END AS reason
FROM promotion_price_by_site
""")
